# Automated License Plate Detection using Faster R-CNN and CRNN

This notebook implements an automated license plate detection pipeline
that combines a Faster R-CNN-based object detection model with CRNN for
optical character recognition.

Faster R-CNN effectively localizes license plates, the CRNN component struggles with character recognition accuracy across diverse international plate formats. 

**Execution Environment:** Google Colab (GPU-enabled)

In [ ]:
class LicensePlateRecognizer:
    def __init__(self):
        print("Initializing OCR model...")
        self.reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
        self.valid_patterns = [
            r'^[A-Z]{2}\d{2}[A-Z]{1,2}\d{4}$',  # KA01AB1234
            r'^[A-Z]{2}\d{2}[A-Z]{2}\d{4}$',    # DL01AB1234
            r'^[A-Z]{3}\d{4}$',                  # ABC1234
            r'^[A-Z]{2}\d{4}$',                  # AB1234
            r'^[A-Z]\d{3}[A-Z]{3}$',             # A123ABC
        ]
        print("OCR model initialized")

    def preprocess_image(self, image):
        """Preprocess license plate image"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply CLAHE for contrast enhancement
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)

        # Denoise
        denoised = cv2.fastNlMeansDenoising(enhanced)

        # Adaptive thresholding
        thresh = cv2.adaptiveThreshold(
            denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, 11, 2
        )

        return thresh

    def recognize_plate(self, image):
        """Recognize license plate text"""
        try:
            # Original image
            results1 = self.reader.readtext(image, detail=1)

            # Preprocessed image
            preprocessed = self.preprocess_image(image)
            results2 = self.reader.readtext(preprocessed, detail=1)

            # Combine results
            all_results = results1 + results2

            # Extract best result
            best_text = None
            best_confidence = 0

            for (bbox, text, confidence) in all_results:
                if confidence > 0.3:
                    cleaned = self.clean_text(text)
                    if cleaned and len(cleaned) >= 4:
                        if confidence > best_confidence:
                            best_text = cleaned
                            best_confidence = confidence

            return best_text, best_confidence

        except Exception as e:
            print(f"Recognition error: {e}")
            return None, 0.0

    def clean_text(self, text):
        """Clean and format license plate text"""
        # Remove special characters and spaces
        cleaned = ''.join(c for c in text if c.isalnum()).upper()

        # Fix common OCR mistakes
        replacements = {
            'O': '0', 'I': '1', 'Z': '2', 'S': '5',
            'B': '8', 'G': '6', 'Q': '0'
        }

        # Apply replacements contextually
        result = []
        for i, char in enumerate(cleaned):
            # If surrounded by numbers, likely a number
            prev_is_digit = i > 0 and cleaned[i-1].isdigit()
            next_is_digit = i < len(cleaned)-1 and cleaned[i+1].isdigit()

            if (prev_is_digit or next_is_digit) and char in replacements:
                result.append(replacements[char])
            else:
                result.append(char)

        return ''.join(result)

    def is_valid_format(self, plate_text):
        """Check if plate matches valid patterns"""
        if not plate_text or len(plate_text) < 4:
            return False

        for pattern in self.valid_patterns:
            if re.match(pattern, plate_text):
                return True

        # Accept if has reasonable mix of letters and numbers
        has_letters = any(c.isalpha() for c in plate_text)
        has_numbers = any(c.isdigit() for c in plate_text)
        reasonable_length = 4 <= len(plate_text) <= 12

        return has_letters and has_numbers and reasonable_length